In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-19 00:48:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.43MB/s    in 0.2s    

2022-08-19 00:48:26 (5.43 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

#### EXTRACT


In [ ]:
# Read in data from S3 Bukets
from pyspark import SparkFiles

url= "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Beauty_v1_00.tsv.gz'), sep='\t', header=True, inferSchema = True)
df.show(n=20,truncate=False)


+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                                                 |product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline              

In [ ]:
df2 = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df2.show(10)

# Data fraame with votes equal or greater than 20
df3 = df2.filter(df2["total_votes"] >= 20)
df4 = df3.filter(df3["helpful_votes"]/df3["total_votes"] >= 0.5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            0|          2|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import col, avg

paid_df = df4.filter(df4['vine']== 'Y')
unpaid_df = df4.filter(df4['vine']== 'N')

# Revies part of the vine program
paid_df.describe().show()

# Review that were not part of the vine program
unpaid_df.describe().show()

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|               647|               647|               647| 647|              647|
|   mean|3.8330757341576507| 79.49613601236476| 89.41112828438949|null|             null|
| stddev|1.1537877983967164|148.85237840828043|159.41229447031864|null|             null|
|    min|                 1|                13|                20|   Y|                N|
|    max|                 5|              1778|              1827|   Y|                Y|
+-------+------------------+------------------+------------------+----+-----------------+

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+------

In [ ]:
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)
print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

647
229
0.35394126738794435


In [ ]:
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_non_vine = float(unpaid_five_star_number) / float(unpaid_number)
print(unpaid_number)
print(unpaid_five_star_number)
print(float(unpaid_five_star_number) / float(unpaid_number))

74113
43217
0.5831230688273312
